In [2]:
import pandas as pd
import numpy as np

In [3]:
assumptions_raw = pd.read_excel("Mash3_BGS_Talk_v01.xlsx")

In [4]:
assumptions_raw.head(11)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,MN0,MN1,MN2,MN3,MN4,MN5,MN6,...,MN18,MN19,MN20,MN21,MN22,MN23,MN24,Unnamed: 29,YR1,YR2
0,ASSUMPTIONS,YR1,YR2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Starting Organic / Social Visitors,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Additive Social / Organic per Month,500,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,K-Factor (Users),0.2,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,K-Factor (Subscribers),0.5,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Traffic -> User Conversion Rate,0.25,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Cost of Paid Traffic,2,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Marketing Spend / Month,10000,25000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,MoM Retention (Users),0.3,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,User -> Subscriber Conversion Rate,0.1,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
assumptions = (
    assumptions_raw
    .iloc[1:,0:3]
    .head(10)
    .rename(columns=
            {'Unnamed: 0':'ASSUMPTIONS',
             'Unnamed: 1':'YR1',
             'Unnamed: 2': 'YR2'})
    )
assumptions

,ASSUMPTIONS,YR1,YR2
1,Starting Organic / Social Visitors,1000,NaN
2,Additive Social / Organic per Month,500,1000
3,K-Factor (Users),0.2,0.3
4,K-Factor (Subscribers),0.5,1
5,Traffic -> User Conversion Rate,0.25,0.3
6,Cost of Paid Traffic,2,2.5
7,Marketing Spend / Month,10000,25000
8,MoM Retention (Users),0.3,0.5
9,User -> Subscriber Conversion Rate,0.1,0.15
10,MoM Retention (Subscribers),0.4,0.6


In [6]:
subscribers = pd.DataFrame()
total_months = 25


In [7]:
def apply_assumption(row, assumption_name):
    if row['month'] < 13:
        return assumptions.loc[assumptions['ASSUMPTIONS'] == assumption_name, 'YR1'].iloc[0]
    else:
        return assumptions.loc[assumptions['ASSUMPTIONS'] == assumption_name, 'YR2'].iloc[0]

In [8]:
subscribers['month'] = [month for month in range(0,total_months)]

subscribers['Traffic -> User Conversion Rate'] = subscribers.apply(apply_assumption, axis=1, args=("Traffic -> User Conversion Rate",))
subscribers['K-Factor (Users)'] = subscribers.apply(apply_assumption, axis=1, args=("K-Factor (Users)",))
subscribers['Paid Traffic (Installs / Visitors)'] = subscribers.apply(apply_assumption, axis=1, args=("Cost of Paid Traffic",))
subscribers['Marketing Spend / Month'] = subscribers.apply(apply_assumption, axis=1, args=("Marketing Spend / Month",))
subscribers['MoM Retention (Users)'] = subscribers.apply(apply_assumption, axis=1, args=("MoM Retention (Users)",))  
subscribers['MoM Retention (Subscribers)'] = subscribers.apply(apply_assumption, axis=1, args=("MoM Retention (Subscribers)",))  
subscribers['K-Factor (Subscribers)'] = subscribers.apply(apply_assumption, axis=1, args=("K-Factor (Subscribers)",))
subscribers['User -> Subscriber Conversion Rate'] = subscribers.apply(apply_assumption, axis=1, args=("User -> Subscriber Conversion Rate",))



In [9]:
for i in subscribers['month']:

    # ----------- Organic / Social Visitors --------------
    if i == 0:
        subscribers['Organic / Social Visitors'] = assumptions.loc[assumptions['ASSUMPTIONS'] == "Starting Organic / Social Visitors", 'YR1'].iloc[0]
    elif i < 13:
        subscribers.at[i, 'Organic / Social Visitors'] = subscribers.at[i-1, 'Organic / Social Visitors'] + assumptions.loc[assumptions['ASSUMPTIONS'] == "Additive Social / Organic per Month", 'YR1'].iloc[0]
    else:
        subscribers.at[i, 'Organic / Social Visitors'] = subscribers.at[i-1, 'Organic / Social Visitors'] + assumptions.loc[assumptions['ASSUMPTIONS'] == "Additive Social / Organic per Month", 'YR2'].iloc[0]
    
    subscribers['Cost / New User'] = subscribers['Paid Traffic (Installs / Visitors)'] / subscribers['Traffic -> User Conversion Rate']
    subscribers['New Acquired Users (Paid)'] = subscribers['Marketing Spend / Month'] / subscribers['Cost / New User']

    # ----------- Viral Acquired Visitors 
    if  i == 0:
        subscribers["Viral Acquired Visitors"] = 0
    else:
        subscribers.at[i,"Viral Acquired Visitors"] =  subscribers.at[i-1, '(All Channels) New Users'] * subscribers.at[i-1,'K-Factor (Users)'] + subscribers.at[i-1, '(All) Total Current Subscribers'] * subscribers.at[i-1,'K-Factor (Subscribers)']

    # ----------- New Organic + Viral Visitors
        
    subscribers['New Organic + Viral Visitors'] = (subscribers["Viral Acquired Visitors"] + subscribers['Organic / Social Visitors']) * subscribers['Traffic -> User Conversion Rate']

    subscribers['(All Channels) New Users'] = subscribers['New Acquired Users (Paid)'] + subscribers['New Organic + Viral Visitors']

    if  i == 0:
        subscribers["(All Channels) Retained Users"] = 0
    else:
        subscribers.at[i,"(All Channels) Retained Users"] =  subscribers.at[i-1, "(All Channels) (New + Retained) Users"] * subscribers.at[i-1,'MoM Retention (Users)'] - subscribers.at[i-1, "(New) Subscribers"] 

    if  i == 0:
        subscribers["(All Channels) (New + Retained) Users"] = subscribers['(All Channels) New Users'] + subscribers["(All Channels) Retained Users"]
    else:    
       subscribers.at[i,"(All Channels) (New + Retained) Users"] = subscribers.at[i,'(All Channels) New Users'] + subscribers.at[i,"(All Channels) Retained Users"] - subscribers.at[i-1, "(New) Subscribers"]

    subscribers['(New) Subscribers'] = subscribers['User -> Subscriber Conversion Rate'] * subscribers["(All Channels) (New + Retained) Users"]

    if  i == 0:
        subscribers["(Retained) Subscribers"] = 0
    else:
        subscribers.at[i,"(Retained) Subscribers"] =  subscribers.at[i-1, "(New) Subscribers"] * subscribers.at[i-1,'MoM Retention (Subscribers)']

    subscribers["(All) Total Current Subscribers"] = subscribers["(Retained) Subscribers"] + subscribers['(New) Subscribers']

    subscribers['(All Channels) Cumulative New Users'] = subscribers['(All Channels) New Users'].cumsum()
    

C:\Users\usuario\AppData\Local\Temp\ipykernel_11588\3692095055.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '467.1875' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  subscribers.at[i,"Viral Acquired Visitors"] =  subscribers.at[i-1, '(All Channels) New Users'] * subscribers.at[i-1,'K-Factor (Users)'] + subscribers.at[i-1, '(All) Total Current Subscribers'] * subscribers.at[i-1,'K-Factor (Subscribers)']
C:\Users\usuario\AppData\Local\Temp\ipykernel_11588\3692095055.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '373.75' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  subscribers.at[i,"(All Channels) Retained Users"] =  subscribers.at[i-1, "(All Channels) (New + Retained) Users"] * subscribers.at[i-1,'MoM Retention (Users)'] - subs

In [10]:
subscribers.iloc[:,10:16].head(15)

,Cost / New User,New Acquired Users (Paid),Viral Acquired Visitors,New Organic + Viral Visitors,(All Channels) New Users,(All Channels) Retained Users
0,8.000000,1250.0,0.000000,250.000000,1500.000000,0.000000
1,8.000000,1250.0,375.000000,468.750000,1718.750000,300.000000
2,8.000000,1250.0,467.187500,616.796875,1866.796875,373.750000
3,8.000000,1250.0,513.417969,753.354492,2003.354492,410.734375
4,8.000000,1250.0,552.180420,888.045105,2138.045105,441.744336
5,8.000000,1250.0,589.729318,1022.432330,2272.432330,471.783455
6,8.000000,1250.0,627.081014,1156.770254,2406.770254,501.664811
7,8.000000,1250.0,664.400665,1291.100166,2541.100166,531.520532
8,8.000000,1250.0,701.715108,1425.428777,2675.428777,561.372086
9,8.000000,1250.0,739.028705,1559.757176,2809.757176,591.222964


In [11]:
subscribers['(All) Total Current Subscribers']

0      150.000000
1      246.875000
2      280.117188
3      303.019043
4      324.240594
5      345.189097
6      366.093228
7      386.990150
8      407.885899
9      428.781459
10     449.676987
11     470.572510
12     491.468033
13    1045.394854
14    1701.156040
15    1991.976045
16    2167.622380
17    2301.081282
18    2419.370831
19    2532.198960
20    2643.060989
21    2753.215222
22    2863.114648
23    2972.922344
24    3082.697017
Name: (All) Total Current Subscribers, dtype: float64

In [12]:
import plotly.express as px

In [13]:
px.line(subscribers,
        x = 'month',
        y = "(All) Total Current Subscribers")

In [14]:
df2 = pd.read_excel('Mash3_BGS_Talk_v02b.xlsx')

In [15]:
df2.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,MN0,MN1,MN2,MN3,MN4,MN5,MN6,...,MN18,MN19,MN20,MN21,MN22,MN23,MN24,Unnamed: 29,YR1,YR2
0,ASSUMPTIONS,YR1,YR2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Traffic Sources,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Starting Organic / Social Visitors,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Additive Social / Organic per Month,500,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cost of Paid Traffic,2,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_clean = (
    df2
    .iloc[1:,0:3]
    .head(13)
    .rename(columns=
            {'Unnamed: 0':'ASSUMPTIONS',
             'Unnamed: 1':'YR1',
             'Unnamed: 2': 'YR2'})
    .dropna(subset=['YR1'])
    )
df_clean.head(15)

,ASSUMPTIONS,YR1,YR2
2,Starting Organic / Social Visitors,1000,NaN
3,Additive Social / Organic per Month,500,1000
4,Cost of Paid Traffic,2,2.5
5,Marketing Spend / Month,10000,25000
6,K-Factor (Users),0.2,0.3
7,K-Factor (Subscribers),0.5,1
10,Traffic -> User Conversion Rate,0.25,0.3
11,MoM Retention (Users),0.3,0.5
12,User -> Subscriber Conversion Rate,0.1,0.15
13,MoM Retention (Subscribers),0.4,0.6


In [17]:
px.line(
    subscribers,
    x = 'month',
    y = "(All) Total Current Subscribers",
    title = "Evolution of Subscribers over Time",
    labels = {"(All) Total Current Subscribers" : "Total Subscribers"}
    ).update_layout(
        title_font = dict(size = 35),
        title = {'x':0.5,
                 'y': .94,
                 'xanchor': 'center'} ,
        height = 550
    ).update_xaxes(
        title_font = dict(
            size = 20
        ),
        tickfont_size= 20
    ).update_yaxes(
        title_font = dict(
            size = 20
        ),
        tickfont_size= 15
    ).add_annotation(
        text = "Change in slope due to increase in<br><b>conversion</b> and <b>retention</b>",
        x = 12,
        y = 491.468,
        arrowhead = 1,
        yshift=10,
        xanchor="right",
        font=dict(
            size=17
            ),
        arrowsize=2,
        align="center",       
    )

In [18]:
fig = px.line(
    subscribers,
    x='month',
    y="(All) Total Current Subscribers",
    title="Evolution of Subscribers over Time",
    labels={"(All) Total Current Subscribers": "Total Subscribers"}
)

fig.update_layout(
    plot_bgcolor='white',  
    paper_bgcolor='white',
    title_font=dict(size=35),
    title={'x':0.5, 'y':0.96, 'xanchor': 'center'},
    height=800,  
    width=1200, 
).update_xaxes(
    title_font=dict(size=20),
    tickfont_size=20,
    showgrid=False 
).update_yaxes(
    title_font=dict(size=20),
    tickfont_size=15,
    showgrid=False
).add_annotation(
    text="Change in slope due to increase in<br><b>conversion</b>, <b>retention</b> and <b>ad spend</b>",
    x=12,
    y=491.468,
    arrowhead=1,
    yshift=10,
    xanchor="right",
    showarrow=True,
    font=dict(size=20),
    arrowsize=2,
    align="center",
    ax=-50,  
    ay=-30
)

fig.add_annotation(
    text="Projected subscriber growth over the first two years based on business model simulations,<br>with product improvements leading to increased retention and conversion rates.",
    xref="paper", yref="paper",
    x=0.5, y=0.96,  
    showarrow=False,
    font=dict(size=20, color="grey"),
    xanchor='center', yanchor='bottom',
)

fig.update_layout(template='plotly_white') 


fig.show()

In [19]:

def subscription_growth(time_frame, assumptions, retention_subscribers = 0):

    subscribers = pd.DataFrame()

    subscribers['month'] = range(0,time_frame)

    if retention_subscribers == 0:
        subscribers['MoM Retention (Subscribers)'] = subscribers.apply(apply_assumption, axis=1, args=("MoM Retention (Subscribers)",))  
    else:
        subscribers['MoM Retention (Subscribers)'] = retention_subscribers

    subscribers['Traffic -> User Conversion Rate'] = subscribers.apply(apply_assumption, axis=1, args=("Traffic -> User Conversion Rate",))
    subscribers['K-Factor (Users)'] = subscribers.apply(apply_assumption, axis=1, args=("K-Factor (Users)",))
    subscribers['Paid Traffic (Installs / Visitors)'] = subscribers.apply(apply_assumption, axis=1, args=("Cost of Paid Traffic",))
    subscribers['Marketing Spend / Month'] = subscribers.apply(apply_assumption, axis=1, args=("Marketing Spend / Month",))
    subscribers['MoM Retention (Users)'] = subscribers.apply(apply_assumption, axis=1, args=("MoM Retention (Users)",))  
    subscribers['K-Factor (Subscribers)'] = subscribers.apply(apply_assumption, axis=1, args=("K-Factor (Subscribers)",))
    subscribers['User -> Subscriber Conversion Rate'] = subscribers.apply(apply_assumption, axis=1, args=("User -> Subscriber Conversion Rate",))

    for i in subscribers['month']:

        # ----------- Organic / Social Visitors --------------
        if i == 0:
            subscribers['Organic / Social Visitors'] = assumptions.loc[assumptions['ASSUMPTIONS'] == "Starting Organic / Social Visitors", 'YR1'].iloc[0]
        elif i < 13:
            subscribers.at[i, 'Organic / Social Visitors'] = subscribers.at[i-1, 'Organic / Social Visitors'] + assumptions.loc[assumptions['ASSUMPTIONS'] == "Additive Social / Organic per Month", 'YR1'].iloc[0]
        else:
            subscribers.at[i, 'Organic / Social Visitors'] = subscribers.at[i-1, 'Organic / Social Visitors'] + assumptions.loc[assumptions['ASSUMPTIONS'] == "Additive Social / Organic per Month", 'YR2'].iloc[0]
        
        subscribers['Cost / New User'] = subscribers['Paid Traffic (Installs / Visitors)'] / subscribers['Traffic -> User Conversion Rate']
        subscribers['New Acquired Users (Paid)'] = subscribers['Marketing Spend / Month'] / subscribers['Cost / New User']

        # ----------- Viral Acquired Visitors 
        if  i == 0:
            subscribers["Viral Acquired Visitors"] = 0
        else:
            subscribers.at[i,"Viral Acquired Visitors"] =  subscribers.at[i-1, '(All Channels) New Users'] * subscribers.at[i-1,'K-Factor (Users)'] + subscribers.at[i-1, '(All) Total Current Subscribers'] * subscribers.at[i-1,'K-Factor (Subscribers)']

        # ----------- New Organic + Viral Visitors
            
        subscribers['New Organic + Viral Visitors'] = (subscribers["Viral Acquired Visitors"] + subscribers['Organic / Social Visitors']) * subscribers['Traffic -> User Conversion Rate']

        subscribers['(All Channels) New Users'] = subscribers['New Acquired Users (Paid)'] + subscribers['New Organic + Viral Visitors']

        if  i == 0:
            subscribers["(All Channels) Retained Users"] = 0
        else:
            subscribers.at[i,"(All Channels) Retained Users"] =  subscribers.at[i-1, "(All Channels) (New + Retained) Users"] * subscribers.at[i-1,'MoM Retention (Users)'] - subscribers.at[i-1, "(New) Subscribers"] 

        if  i == 0:
            subscribers["(All Channels) (New + Retained) Users"] = subscribers['(All Channels) New Users'] + subscribers["(All Channels) Retained Users"]
        else:    
            subscribers.at[i,"(All Channels) (New + Retained) Users"] = subscribers.at[i,'(All Channels) New Users'] + subscribers.at[i,"(All Channels) Retained Users"] - subscribers.at[i-1, "(New) Subscribers"]

        subscribers['(New) Subscribers'] = subscribers['User -> Subscriber Conversion Rate'] * subscribers["(All Channels) (New + Retained) Users"]

        if  i == 0:
            subscribers["(Retained) Subscribers"] = 0
        else:
            subscribers.at[i,"(Retained) Subscribers"] =  subscribers.at[i-1, "(New) Subscribers"] * subscribers.at[i-1,'MoM Retention (Subscribers)']

        subscribers["(All) Total Current Subscribers"] = subscribers["(Retained) Subscribers"] + subscribers['(New) Subscribers']

        subscribers['(All Channels) Cumulative New Users'] = subscribers['(All Channels) New Users'].cumsum()

    return subscribers

In [20]:
subscription25 = subscription_growth(time_frame = 24, assumptions = assumptions, retention_subscribers = 0.25)
subscription65 = subscription_growth(time_frame = 24, assumptions = assumptions, retention_subscribers = 0.65)
subscription65.head()

C:\Users\usuario\AppData\Local\Temp\ipykernel_11588\2869945529.py:60: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '37.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

C:\Users\usuario\AppData\Local\Temp\ipykernel_11588\2869945529.py:37: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '455.9375' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

C:\Users\usuario\AppData\Local\Temp\ipykernel_11588\2869945529.py:48: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '373.75' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

C:\Users\usuario\AppData\Local\Temp\ipykernel_11588\2869945529.py:60: FutureWarning:

Setting an item of incompa

,month,MoM Retention (Subscribers),Traffic -> User Conversion Rate,K-Factor (Users),Paid Traffic (Installs / Visitors),Marketing Spend / Month,MoM Retention (Users),K-Factor (Subscribers),User -> Subscriber Conversion Rate,Organic / Social Visitors,...,New Acquired Users (Paid),Viral Acquired Visitors,New Organic + Viral Visitors,(All Channels) New Users,(All Channels) Retained Users,(All Channels) (New + Retained) Users,(New) Subscribers,(Retained) Subscribers,(All) Total Current Subscribers,(All Channels) Cumulative New Users
0,0,0.65,0.25,0.2,2.0,10000,0.3,0.5,0.1,1000,...,1250.0,0.000000,250.000000,1500.000000,0.000000,1500.000000,150.000000,0.000000,150.000000,1500.000000
1,1,0.65,0.25,0.2,2.0,10000,0.3,0.5,0.1,1500,...,1250.0,375.000000,468.750000,1718.750000,300.000000,1868.750000,186.875000,97.500000,284.375000,3218.750000
2,2,0.65,0.25,0.2,2.0,10000,0.3,0.5,0.1,2000,...,1250.0,485.937500,621.484375,1871.484375,373.750000,2058.359375,205.835938,121.468750,327.304688,5090.234375
3,3,0.65,0.25,0.2,2.0,10000,0.3,0.5,0.1,2500,...,1250.0,537.949219,759.487305,2009.487305,411.671875,2215.323242,221.532324,133.793359,355.325684,7099.721680
4,4,0.65,0.25,0.2,2.0,10000,0.3,0.5,0.1,3000,...,1250.0,579.560303,894.890076,2144.890076,443.064648,2366.422400,236.642240,143.996011,380.638251,9244.611755


In [21]:
import plotly.graph_objects as go


fig = px.line(
    subscribers,
    x='month',
    y="(All) Total Current Subscribers",
    title="Evolution of Subscribers over Time",
    labels={
        "(All) Total Current Subscribers": "Total Subscribers",
        "month": "Month"}
)

fig.update_layout(
    plot_bgcolor='white',  
    paper_bgcolor='white',
    title_font=dict(size=35),
    title={'x':0.5, 'y':0.96, 'xanchor': 'center'},
    height=800,  
    width=1200,
    showlegend=False 
).update_xaxes(
    title_font=dict(size=20),
    tickfont_size=20,
    showgrid=False 
).update_yaxes(
    title_font=dict(size=20),
    tickfont_size=15,
    showgrid=False
).add_annotation(
    text="Change in slope due to increase in<br><b>conversion</b>, <b>retention</b> and <b>ad spend</b>",
    x=12,
    y=subscribers.loc[12, "(All) Total Current Subscribers"],
    arrowhead=1,
    yshift=10,
    xanchor="right",
    showarrow=True,
    font=dict(size=20),
    arrowsize=2,
    align="center",
    ax=-50,  
    ay=-30
)

fig.add_annotation(
    text="Projected subscriber growth over the first two years based on business model simulations,<br>with product improvements leading to increased retention and conversion rates.",
    xref="paper", yref="paper",
    x=0.5, y=0.96,  
    showarrow=False,
    font=dict(size=20, color="grey"),
    xanchor='center', yanchor='bottom',
)

#fig.update_layout(template='plotly_white') 

# Agrega la línea para el límite inferior de la incertidumbre
fig.add_trace(go.Scatter(
    x=subscription25['month'],
    y=subscription25['(All) Total Current Subscribers'],
    mode='lines',
    line=dict(width=0),
    showlegend=False
))

# Agrega la línea para el límite superior de la incertidumbre
fig.add_trace(go.Scatter(
    x=subscription65['month'],
    y=subscription65['(All) Total Current Subscribers'],
    mode='lines',
    fill='tonexty',  # Esto rellena el área entre la línea superior y la línea inferior
    fillcolor='rgba(173, 216, 230, 0.5)',  # Un color azul claro con transparencia
    line=dict(width=0),
    showlegend=False
))

fig.add_annotation(
    text="The area represent all the posible growth<br>"
         "based on the different retentions present<br>"
         "in the industry: <b>25 %</b> to <b>65 %</b> based on research.",
    x=18,
    y=2500,
    arrowhead=1,
    yshift=10,
    xanchor="right",
    showarrow=True,
    font=dict(size=20),
    arrowsize=2,
    align="center",
    ax=-50,  
    ay=-30
)

fig.show()

In [22]:
subscribers.to_csv('subscription_simulation_data.csv')

In [23]:

fig2 = px.scatter(
    subscribers,
    x='Marketing Spend / Month', 
    y='(New) Subscribers', 
    trendline='ols',  
    title='Marketing Spend vs. Subscriber Growth'
)
fig2.update_layout(
    plot_bgcolor='white',  
    paper_bgcolor='white',
    title_font=dict(size=35),
    title={'x':0.5, 'y':0.96, 'xanchor': 'center'},
    height=800,  
    width=1200,
    showlegend=False 
)

fig2.add_annotation(
    text="Expected subscriber growth based on proyected marketing spend year over year.",
    xref="paper", yref="paper",
    x=0.5, y=1.02,  
    showarrow=False,
    font=dict(size=20, color="grey"),
    xanchor='center', yanchor='bottom',
)

fig2.add_annotation(
    text="While the exact relationship between <b>marketing spend</b> and<br>"
         "its influence on subscriber growth is not fully known,<br>"
         "marketing spend is expected to be one of the main factors<br>"
         "affecting the <b>volume of users</b>.",
    x=22000,
    y=1500,
    arrowhead=1,
    yshift=10,
    xanchor="right",
    showarrow=False,
    font=dict(size=20),
    arrowsize=2,
    align="center",
    ax=-50,  
    ay=-30
)

fig2.update_layout(
    xaxis_title='Marketing Spend per Month',
    yaxis_title='Total Subscribers',
    title_font=dict(size=24),
    xaxis_tickfont_size=16,
    yaxis_tickfont_size=16
)


fig2.show()

In [24]:
subscribers_prueba = subscription_growth(time_frame = 25, assumptions = assumptions)

C:\Users\usuario\AppData\Local\Temp\ipykernel_11588\2869945529.py:37: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '467.1875' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

C:\Users\usuario\AppData\Local\Temp\ipykernel_11588\2869945529.py:48: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '373.75' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

C:\Users\usuario\AppData\Local\Temp\ipykernel_11588\2869945529.py:60: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '74.75' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



In [25]:
subscribers_prueba.tail()

,month,MoM Retention (Subscribers),Traffic -> User Conversion Rate,K-Factor (Users),Paid Traffic (Installs / Visitors),Marketing Spend / Month,MoM Retention (Users),K-Factor (Subscribers),User -> Subscriber Conversion Rate,Organic / Social Visitors,...,New Acquired Users (Paid),Viral Acquired Visitors,New Organic + Viral Visitors,(All Channels) New Users,(All Channels) Retained Users,(All Channels) (New + Retained) Users,(New) Subscribers,(Retained) Subscribers,(All) Total Current Subscribers,(All Channels) Cumulative New Users
20,20,0.6,0.3,0.3,2.5,25000,0.5,1.0,0.15,15000,...,3000.0,5134.147493,6040.244248,9040.244248,3753.821026,11185.284834,1677.792725,965.268264,2643.060989,91937.460255
21,21,0.6,0.3,0.3,2.5,25000,0.5,1.0,0.15,16000,...,3000.0,5355.134263,6406.540279,9406.540279,3914.849692,11643.597246,1746.539587,1006.675635,2753.215222,101344.000534
22,22,0.6,0.3,0.3,2.5,25000,0.5,1.0,0.15,17000,...,3000.0,5575.177306,6772.553192,9772.553192,4075.259036,12101.272641,1815.190896,1047.923752,2863.114648,111116.553725
23,23,0.6,0.3,0.3,2.5,25000,0.5,1.0,0.15,18000,...,3000.0,5794.880606,7138.464182,10138.464182,4235.445424,12558.718710,1883.807806,1089.114538,2972.922344,121255.017907
24,24,0.6,0.3,0.3,2.5,25000,0.5,1.0,0.15,19000,...,3000.0,6014.461599,7504.338480,10504.338480,4395.551548,13016.082222,1952.412333,1130.284684,3082.697017,131759.356387


In [125]:
channels_for_new_users = px.bar(
    subscribers.rename(columns={
        'Organic / Social Visitors': 'Organic / Social Users',
        'Viral Acquired Visitors' : 'Viral Acquired Users',
        'New Acquired Users (Paid)': 'New Acquired Subscribers'
        }),
    x='month', 
    y=['Organic / Social Users','Viral Acquired Users','New Acquired Subscribers'],
    title='Sources of New Users'
).update_layout(
    plot_bgcolor='white',  
    paper_bgcolor='white',
    title_font=dict(size=35),
    title={'x':0.5, 'y':0.96, 'xanchor': 'center'},
    height=800,  
    width=1200,
    showlegend=True,
    xaxis_title='Month',
    yaxis_title='Total New Users',
    xaxis_tickfont_size=16,
    yaxis_tickfont_size=16,
    legend=dict(
    yanchor="top",
    y=0.98,
    xanchor="left",
    x=0.01),
    legend_title_text='Channel'
).add_shape(
    name="first stage",
    showlegend=False,
    type="rect",
    xref="paper",
    line=dict(dash="dash"),
    x0=0.0,
    x1=0.12,
    y0=0,
    y1=5000,
).add_annotation(
    text =  "Initially, the majority of traffic<br>"
            "would likely come from<br>"
            "paid marketing which the<br>"
            "business controls.",
    x = 3.5,
    y = 6500,
    showarrow=False,
    font=dict(size=18),
    yshift=35
).add_annotation(
    text =  "As the platform evolves, we'd expect<br>"
            "customers to shift to organic / social traffic<br>"
            "based on the effect of viral growth",
    x = 15,
    y = 25000,
    showarrow=False,
    font=dict(size=18),
    yshift=35
).add_annotation(
    text="Expected traffic of new users coming from marketing spend, virality, and organic visitors.",
    xref="paper", yref="paper",
    x=0.5, y=1.01,  
    showarrow=False,
    font=dict(size=20, color="grey"),
    xanchor='center', yanchor='bottom',
)


channels_for_new_users.show()


c:\Users\usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [145]:
import plotly.graph_objects as go

nodos_x = [1, 2, 3]
nodos_y = [3, 3, 3]
nodos_texto = ["New Acquired Users (Paid)", "Organic / Social Visitors", "Viral Acquired Visitors"]

nodo_new_users_x = [2]
nodo_new_users_y = [1]
nodo_new_users_text = ["New Users"]

conexiones_x = [nodos_x[0], nodo_new_users_x[0], None,  
                nodos_x[1], nodo_new_users_x[0], None,  
                nodos_x[2], nodo_new_users_x[0], None]  
conexiones_y = [nodos_y[0], nodo_new_users_y[0], None,  
                nodos_y[1], nodo_new_users_y[0], None,
                nodos_y[2], nodo_new_users_y[0], None]

fig = go.Figure(go.Scatter(x=nodos_x, y=nodos_y, mode='markers+text', 
                           text=nodos_texto, textposition='top center', 
                           marker=dict(size=25, color='LightSkyBlue'), name='Nodos',textfont=dict(size=16)))

fig.add_trace(go.Scatter(x=conexiones_x, y=conexiones_y, mode='lines', line=dict(color='royalblue', width=2), name='Conexiones'))
fig.add_trace(go.Scatter(x=[2, 1.5], y=[1, -1], mode='lines', line=dict(color='tomato', width=2), name='Conexiones'))
fig.add_trace(go.Scatter(x=[2, 2.5], y=[1, -1], mode='lines', line=dict(color='springgreen', width=2), name='Conexiones'))
fig.add_trace(go.Scatter(x=[2.5, 2], y=[-1, -3], mode='lines', line=dict(color='tomato', width=2), name='Conexiones'))
fig.add_trace(go.Scatter(x=[2.5, 3], y=[-1, -3], mode='lines', line=dict(color='springgreen', width=2), name='Conexiones'))

fig.update_layout(title="Expected Customer Journey",
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    showlegend=False,    plot_bgcolor='white',  
                    paper_bgcolor='white',
                    title_font=dict(size=35),
                    height=1000,  
                    width=1200,)

fig.update_layout(title={'x':0.5, 'y':0.97, 'xanchor': 'center'})

fig.add_trace(go.Scatter(x=nodo_new_users_x, y=nodo_new_users_y, mode='markers+text', 
                         text=nodo_new_users_text, textposition='middle right',
                         marker=dict(size=25, color='Green'), name='Nodos',textfont=dict(size=16)))

new_nodes_x = [1.5,2.5]
new_nodes_y = [-1, -1]
nodo_nodes_text = ["Churned Users", "New Subscribers"]

fig.add_trace(go.Scatter(x=new_nodes_x, y=new_nodes_y, mode='markers+text', 
                         text=nodo_nodes_text, textposition='middle left',
                         marker=dict(size=25, color=['Red', 'green']), name='Nodos',textfont=dict(size=16)))

final_nodes_x = [2,3]
final_nodes_y = [-3, -3]
final_nodes_text = ["Churned Subscribers", "Retained Subscribers"]

fig.add_trace(go.Scatter(x=final_nodes_x, y=final_nodes_y, mode='markers+text', 
                         text=final_nodes_text, textposition='middle left',
                         marker=dict(size=25, color=['Red', 'green']), name='Nodos',textfont=dict(size=16)))

fig.add_annotation(
    text="Expected Customer Journey Based on Research of the Current State of the E-Learning Industry.",
    xref="paper", yref="paper",
    x=0.5, y=1.01,  
    showarrow=False,
    font=dict(size=20, color="grey"),
    xanchor='center', yanchor='bottom',
)

fig.add_annotation(
    text="According to industry statistics<br><b>Retention Rates</b> can<br>be between <b>25%</b> to <b>60%</b>.",
    xref="paper", yref="paper",
    x=0.85, y=0.40,  
    showarrow=False,
    font=dict(size=20),
    xanchor='center', yanchor='bottom',
)

fig.add_annotation(
    text="<b>Virality</b> will come<br>from users and subscribers<br><b>attracting</b> new users.",
    xref="paper", yref="paper",
    x=0.85, y=0.70,  
    showarrow=False,
    font=dict(size=20),
    xanchor='center', yanchor='bottom',
)

fig.add_annotation(
    text="<b>Acquired Users</b> will come<br>from our<br><b>marketing spend</b>.",
    xref="paper", yref="paper",
    x=0.15, y=0.70,  
    showarrow=False,
    font=dict(size=20),
    xanchor='center', yanchor='bottom',
)

fig.show()
